In [1]:
import pandas as pd
from sqlalchemy import create_engine
from copy import deepcopy

movie_metadata = pd.read_csv('movie_metadata.csv')
ratings = pd.read_csv('ratings.csv')
movie_metadata['movie_title'] = movie_metadata.movie_title.str.strip()

In [2]:
for col in ['duration','director_facebook_likes','actor_1_facebook_likes','actor_2_facebook_likes',
            'actor_3_facebook_likes','cast_total_facebook_likes','facenumber_in_poster','budget'] :
    movie_metadata[col] = movie_metadata[col].fillna(movie_metadata[col].median())
for col in ['genres','content_rating','language','country','title_year','aspect_ratio'] :
    movie_metadata[col]=movie_metadata[col].fillna('ukjent')

In [3]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

params = {
    'host': 'XXXX.XXXX.XXXX.XXXX',
    'user': 'ubuntu',
    'port': 5432
}

# Connect and create database, disconnect, and reconnect to the right database
connection = connect(**params)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
try :
    connection.cursor().execute('DROP DATABASE movies;')
except :
    pass
connection.cursor().execute('CREATE DATABASE movies;')
connection.close()

In [4]:
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/movies'
engine = create_engine(connection_string)
movie_metadata.to_sql('metadata', engine, index=False)
ratings.to_sql('ratings', engine, index=False)

In [ ]:
connection = connect(**params, dbname='movies')
cursor = connection.cursor()
cursor.execute("SELECT * FROM metadata LIMIT 1;")
cursor.fetchall()

### Create a view of only the columns we want.

In [4]:
cursor.execute("""SELECT * FROM metadata
                LIMIT 1;""")
cursor.fetchall()
cursor.execute("""CREATE VIEW metadata_select AS
                (SELECT movie_title,
                duration,
                genres,
                content_rating,
                language,
                country,
                title_year,
                aspect_ratio,
                director_facebook_likes,
                actor_1_facebook_likes,
                actor_2_facebook_likes,
                actor_3_facebook_likes,
                cast_total_facebook_likes,
                facenumber_in_poster,
                budget
                FROM metadata);
                """)

NameError: name 'cursor' is not defined

### Go through each feature
Check the distribution of them and group accordingly.

In [10]:
cursor.execute("""SELECT content_rating,COUNT(content_rating)
                FROM metadata_select
                GROUP BY content_rating
                ORDER BY COUNT(content_rating) DESC;""")
cursor.fetchall()

[('R', 2118),
 ('PG-13', 1461),
 ('PG', 701),
 ('ukjent', 303),
 ('Not Rated', 116),
 ('G', 112),
 ('Unrated', 62),
 ('Approved', 55),
 ('TV-14', 30),
 ('TV-MA', 20),
 ('X', 13),
 ('TV-PG', 13),
 ('TV-G', 10),
 ('Passed', 9),
 ('NC-17', 7),
 ('GP', 6),
 ('M', 5),
 ('TV-Y7', 1),
 ('TV-Y', 1)]

In [11]:
cursor.execute("""UPDATE metadata SET content_rating='andre'
                WHERE content_rating NOT IN ('R','PG-13','PG','G');""")
cursor.execute("""SELECT content_rating,COUNT(content_rating)
                FROM metadata
                GROUP BY content_rating
                ORDER BY COUNT(content_rating) DESC;""")
cursor.fetchall()

[('R', 2118), ('PG-13', 1461), ('PG', 701), ('andre', 651), ('G', 112)]

In [12]:
cursor.execute("""SELECT language,COUNT(language)
                FROM metadata
                GROUP BY language
                ORDER BY COUNT(language) DESC;""")
cursor.fetchall()

[('English', 4704),
 ('French', 73),
 ('Spanish', 40),
 ('Hindi', 28),
 ('Mandarin', 26),
 ('German', 19),
 ('Japanese', 18),
 ('ukjent', 12),
 ('Italian', 11),
 ('Cantonese', 11),
 ('Russian', 11),
 ('Korean', 8),
 ('Portuguese', 8),
 ('Swedish', 5),
 ('Danish', 5),
 ('Hebrew', 5),
 ('Arabic', 5),
 ('Polish', 4),
 ('Norwegian', 4),
 ('Dutch', 4),
 ('Persian', 4),
 ('Thai', 3),
 ('Chinese', 3),
 ('Zulu', 2),
 ('None', 2),
 ('Dari', 2),
 ('Icelandic', 2),
 ('Aboriginal', 2),
 ('Indonesian', 2),
 ('Romanian', 2),
 ('Kannada', 1),
 ('Hungarian', 1),
 ('Aramaic', 1),
 ('Slovenian', 1),
 ('Maya', 1),
 ('Bosnian', 1),
 ('Mongolian', 1),
 ('Greek', 1),
 ('Dzongkha', 1),
 ('Filipino', 1),
 ('Panjabi', 1),
 ('Vietnamese', 1),
 ('Urdu', 1),
 ('Swahili', 1),
 ('Kazakh', 1),
 ('Czech', 1),
 ('Tamil', 1),
 ('Telugu', 1)]

In [13]:
cursor.execute("""UPDATE metadata SET language='andre'
                WHERE language != 'English';""")
cursor.execute("""SELECT language,COUNT(language)
                FROM metadata
                GROUP BY language
                ORDER BY COUNT(language) DESC;""")
cursor.fetchall()

[('English', 4704), ('andre', 339)]

In [14]:
cursor.execute("""SELECT country,COUNT(country)
                FROM metadata
                GROUP BY country
                ORDER BY COUNT(country) DESC;""")
cursor.fetchall()

[('USA', 3807),
 ('UK', 448),
 ('France', 154),
 ('Canada', 126),
 ('Germany', 97),
 ('Australia', 55),
 ('India', 34),
 ('Spain', 33),
 ('China', 30),
 ('Japan', 23),
 ('Italy', 23),
 ('Mexico', 17),
 ('Hong Kong', 17),
 ('New Zealand', 15),
 ('South Korea', 14),
 ('Ireland', 12),
 ('Denmark', 11),
 ('Russia', 11),
 ('Brazil', 8),
 ('Norway', 8),
 ('South Africa', 8),
 ('Sweden', 6),
 ('Netherlands', 5),
 ('ukjent', 5),
 ('Poland', 5),
 ('Thailand', 5),
 ('Iran', 4),
 ('Argentina', 4),
 ('Israel', 4),
 ('Romania', 4),
 ('Belgium', 4),
 ('Iceland', 3),
 ('Czech Republic', 3),
 ('West Germany', 3),
 ('Switzerland', 3),
 ('Taiwan', 2),
 ('Greece', 2),
 ('Hungary', 2),
 ('Nigeria', 1),
 ('Afghanistan', 1),
 ('Egypt', 1),
 ('Pakistan', 1),
 ('Slovenia', 1),
 ('Cameroon', 1),
 ('New Line', 1),
 ('Colombia', 1),
 ('Finland', 1),
 ('Libya', 1),
 ('Panama', 1),
 ('Bulgaria', 1),
 ('Cambodia', 1),
 ('Official site', 1),
 ('Aruba', 1),
 ('United Arab Emirates', 1),
 ('Georgia', 1),
 ('Indonesia'

In [15]:
cursor.execute("""UPDATE metadata SET country='andre'
                WHERE country NOT IN ('USA','UK','France','Canada');""")
cursor.execute("""SELECT country,COUNT(country)
                FROM metadata
                GROUP BY country
                ORDER BY COUNT(country) DESC;""")
cursor.fetchall()

[('USA', 3807), ('andre', 508), ('UK', 448), ('France', 154), ('Canada', 126)]

In [16]:
cursor.execute("""SELECT ratings.critic,ratings.audience,metadata_select.*
                FROM ratings
                LEFT JOIN metadata_select ON ratings.movie_title=metadata_select.movie_title""")

In [17]:
total_table = pd.DataFrame(cursor.fetchall())

In [18]:
total_table.columns = ['critic','audience','title','runtime','genres','rating','language','country','year',
                       'aspect_ratio','director_fb','actor_1_fb','actor_2_fb','actor_3_fb','cast_fb','post_face',
                      'budget']

### The 'ugh, I couldn't do that (easily) in SQL' markdown
Fill numerical nans with median (done at start of notebook for ease's sake) <br />
Organising years by decade/old (below) <br />
Dealing with aspect datatype (below) <br />

First, years and aspect ratios aren't number numbers!

In [20]:
total_table['year'] = total_table.year.astype(str)
total_table['aspect_ratio'] = total_table.aspect_ratio.astype(str)

Filter out low representation aspect ratios.

In [22]:
for i in range(len(total_table)) :
    if total_table.loc[i,'aspect_ratio'] not in ['2.35','1.85'] :
        total_table.loc[i,'aspect_ratio'] = 'andre'

Sort movies by decade.

In [23]:
by_decade = []
for year in list(total_table.year) :
    if year == 'ukjent' :
        by_decade.append('ukjent')
    elif int(float(year))<1970 :
        by_decade.append('old')
    else :
        by_decade.append(year[2]+'0s')
total_table['decade'] = by_decade

Create columns for each genre.

In [50]:
genre = deepcopy(total_table)

In [51]:
genre['genres'] = (genre
                    .genres
                    .str.split('|')).to_frame()

In [53]:
each_genre = (genre
            # create a Series of genres for each row; make these the rows in a new df
            .apply(lambda x : pd.Series(x['genres']),axis=1)
            #makes a multiindex object such that the multiple genres are now in one column but have the same index
            .stack()
            #resets the index
            .reset_index(level=1))

In [57]:
total_table = total_table.drop('genres',axis = 1).join(each_genre).drop('level_1',axis=1).rename(columns ={0 : 'genre'})

In [74]:
genre_table = deepcopy(total_table)

In [75]:
genre_table['yes'] = 1

In [78]:
genre_table = genre_table.pivot(columns='genre',values='yes')

Drop duplicates just in case and then save a table with critic scores and another with audience scores.

In [87]:
total_table = total_table.drop_duplicates(subset='title',keep='first').join(fear).fillna(0)

In [89]:
critic_table = total_table.drop('audience',axis=1)

In [90]:
audience_table = total_table.drop('critic',axis=1)

In [91]:
critic_table.to_pickle('./critic_table.csv')

In [92]:
audience_table.to_pickle('./audience_table.csv')